<a href="https://colab.research.google.com/github/8johan/ML-PROJ/blob/main/unet_imp%3B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import torch 
from __future__ import print_function, division
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch
def double_conv(data , intc,outc):
  convo=nn.Sequential(
            nn.Conv2d(intc, outc, kernel_size=3, stride=1, padding=0, bias=True),
            #nn.BatchNorm2d(outc),
            #nn.ReLU(inplace=True),
            nn.Conv2d(outc, outc, kernel_size=3, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(outc),
            nn.ReLU(inplace=True))
  x=convo(data)
  return x
def upconv(data , intc, outc):
  upco=nn.ConvTranspose2d(intc, outc, kernel_size=2, stride=2, padding=0)
  x=upco(data)
  return x 



In [3]:
class countarctive_path(nn.Module):
  def __init__(self) :
    
    super().__init__()
    self.l=[]
    self.maxpoll=nn.MaxPool2d(kernel_size=2, stride=2)
  def forward(self,data):
    x=double_conv(data,3,64)
    self.l.append(x)
    x=self.maxpoll(x)
    self.l.append(x)
    print(x.size())
    x=double_conv(x,64,128)
    x=self.maxpoll(x)
    self.l.append(x)
    print(x.size())
    x=double_conv(x,128,256)
    x=self.maxpoll(x)
    self.l.append(x)
    print(x.size())
    x=double_conv(x,256,512)
    x=self.maxpoll(x)
    self.l.append(x)
    print(x.size())
    x=double_conv(x,512,1024)
  
    self.l.append(x)
  
    return x

  




In [16]:
class expansive_path(nn.Module):
  def __init(self,x):
    super().__init__()
    self.l=x

  def forward(self , data ,l):
    x=upconv(data , 1024 ,512)
    x=torch.cat([x, l[3]], axis=1)
    x=double_conv(x ,1024 ,512)
    x=upconv(x, 512 ,256)
    x=torch.cat([x, l[2]], axis=1)
    x=double_conv(x ,512 ,256)
    x=upconv(x , 256,128)
    x=torch.cat([x, l[1]], axis=1)
    x=double_conv(x ,265 ,128)
    x=upconv(x , 128 ,64)
    x=torch.cat([x, l[0]], axis=1)
    x=double_conv(x ,128 ,64)
    x=upconv(x , 64,2)
    print(x.size())
    return x 
   
    




In [27]:
inputs = torch.randn((2, 3, 572, 572))
model = countarctive_path()
y = model.forward(inputs)
l= model.l
print(y.shape)


torch.Size([2, 64, 284, 284])
torch.Size([2, 128, 140, 140])
torch.Size([2, 256, 68, 68])
torch.Size([2, 512, 32, 32])
torch.Size([2, 1024, 28, 28])


In [28]:
l[3].size()

torch.Size([2, 512, 32, 32])

In [29]:
x=upconv(y , 1024 ,512)

In [30]:
x.size()

torch.Size([2, 512, 56, 56])

In [18]:
model2=expansive_path()
y=model2.forward(y,l)


RuntimeError: ignored